<a href="https://colab.research.google.com/github/Nyamadie/Anomaly-detection/blob/main/Anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash plotly scikit-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()

Saving patient_data (11).csv to patient_data (11).csv


In [7]:
pd.read_csv('patient_data (11).csv')

,filenames,strain,mean,std_dev,z_score,mean_z_score,std_dev_z_score,Data QA,XCMPlus (zany-dust-13457),XCMPlus (radiant-sweep-1),...,InceptionTimePlus (worldly-sweep-243),InceptionTimePlus (zesty-dust-10100),InceptionTimePlus (daily-sweep-188),InceptionTimePlus (glamorous-sweep-103),MultiInceptionTimePlus (peachy-darkness-6716),MultiInceptionTimePlus (eager-breeze-5002),MultiInceptionTimePlus (vital-sunset-5008),MultiInceptionTimePlus (proud-fire-5143),MultiInceptionTimePlus (comic-morning-5149),MultiInceptionTimePlus (super-dew-6656)
0,AA 05.28.21.2.csv,16.0,2.71,1.71,0.21,0.21,1.32,0.54,1.46,5.29,...,0.81,4.33,NaN,NaN,0.58,NaN,NaN,NaN,NaN,3.9
1,AA 05.28.21.1.csv,16.0,0.01,0.00,NaN,-1.40,-0.94,0.08,0.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA 05.28.21.3.csv,16.0,1.51,1.33,-0.63,-0.50,0.82,0.54,0.34,1.89,...,0.62,2.92,NaN,NaN,0.58,NaN,NaN,NaN,NaN,3.9
3,AG 07.06.21.1.csv,19.0,3.42,1.14,0.94,0.64,0.57,0.62,5.78,3.79,...,3.45,3.89,3.16,3.38,NaN,NaN,NaN,NaN,NaN,2.2
4,AG 07.06.21.2.csv,19.0,4.36,1.32,1.52,1.20,0.80,0.62,3.06,4.14,...,4.87,6.58,5.83,3.95,NaN,NaN,NaN,NaN,NaN,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,VE 03.04.21.3.csv,19.0,1.73,0.44,-1.42,-0.37,-0.36,0.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.11,1.57,2.27,1.97,NaN
413,XS 03.04.21.3.csv,21.0,3.68,0.46,2.88,0.79,-0.33,0.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.26,3.82,3.65,2.97,NaN
414,YR 03.03.21.3.csv,13.0,2.10,0.20,-1.27,-0.15,-0.67,0.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.94,1.87,2.24,2.35,NaN
415,YY2 03.02.21.4.csv,21.0,3.82,1.44,1.02,0.87,0.96,0.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.64,2.16,5.07,5.41,NaN


# new1

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import collections

# Initialize the Dash app
app = dash.Dash(__name__)

# Load and preprocess data
def load_data(pickle_file_path, meta_file_path, patient_data_path):
    with open(pickle_file_path, 'rb') as file:
        signal_data = pickle.load(file)

    meta_df = pd.read_excel(meta_file_path)
    df = pd.read_csv(patient_data_path)

    filtered_df = df[df['Data QA'] >= 0.4]
    filtered_df = filtered_df.dropna(subset=['z_score'])
    filtered_df['Signal Quality'] = filtered_df['z_score'].apply(lambda x: 'Worst' if x > 0 else 'Best')

    ppg_signals_df = signal_data['file_data']
    worst_signals = filtered_df[filtered_df['Signal Quality'] == 'Worst']['filenames'].tolist()
    best_signals = filtered_df[filtered_df['Signal Quality'] == 'Best']['filenames'].tolist()

    return ppg_signals_df, worst_signals, best_signals, filtered_df

def normalize_signal(signal):
    return (signal - np.mean(signal)) / np.std(signal)

def extract_features(signal, window_size, stride):
    features = []
    for i in range(0, len(signal) - window_size + 1, stride):
        window = signal[i:i + window_size]
        features.append([
            np.mean(window),      # Mean
            np.std(window),       # Standard deviation
            np.max(window),       # Maximum value
            np.min(window),       # Minimum value
            np.ptp(window),       # Peak-to-peak amplitude
            np.median(window)     # Median
        ])
    return np.array(features)


<IPython.core.display.Javascript object>

In [ ]:
# Load your data
pickle_file_path = ""
meta_file_path = ''
patient_data_path = 'patient_data (11).csv'

ppg_signals_df, worst_signals, best_signals, filtered_df = load_data(
    pickle_file_path, meta_file_path, patient_data_path)

In [ ]:
# Prepare signals for analysis
window_size = 50
stride = 25

# Prepare features for ML models
def extract_group_features(signals, label):
    features = []
    labels = []
    signal_indices = []
    for idx, filename in enumerate(signals):
        signal = normalize_signal(ppg_signals_df[filename]['cycle'].values)
        signal_features = extract_features(signal, window_size, stride)
        features.append(signal_features)
        labels.extend([label] * len(signal_features))
        signal_indices.extend([idx] * len(signal_features))

    return np.vstack(features), np.array(labels), np.array(signal_indices)

# Extract features for "Best" and "Worst" signals
features_best, labels_best, indices_best = extract_group_features(best_signals, 0)  # Label '0' for Best
features_worst, labels_worst, indices_worst = extract_group_features(worst_signals, 1)  # Label '1' for Worst

# Combine the features from both groups
features = np.vstack([features_best, features_worst])
labels = np.concatenate([labels_best, labels_worst])
signal_indices = np.concatenate([indices_best, indices_worst])

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Split data for classification
X_train, X_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.3, random_state=42)

# Initialize models
iso_forest = IsolationForest(contamination=0.1, random_state=42).fit(scaled_features)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train, y_train)

# PCA for visualization
pca = PCA(n_components=2)
pca_features = pca.fit_transform(scaled_features)

# KMeans Clustering
kmeans = KMeans(n_clusters=3, random_state=42).fit(scaled_features)
clusters = kmeans.labels_

In [ ]:
# Updated Dash Layout
app.layout = html.Div([
    html.H1("PPG Signal Analysis Dashboard", style={'textAlign': 'center'}),
    html.H3("Signal Analysis (Best vs Worst)", style={'textAlign': 'center'}),

    # Controls section at the top
    html.Div([
        html.Div([
            html.Label('Select Signal Group'),
            dcc.Dropdown(
                id='group-selector',
                options=[
                    {'label': 'Best Signals', 'value': 'Best'},
                    {'label': 'Worst Signals', 'value': 'Worst'}
                ],
                value='Best',
                style={'width': '200px'}
            )
        ], style={'marginRight': '20px'}),

        html.Div([
            html.Label('Number of Signals to Display'),
            dcc.Slider(
                id='num-signals-slider',
                min=1,
                max=20,
                step=1,
                value=5,
                marks={i: str(i) for i in range(0, 21, 5)},
                tooltip={'placement': 'bottom', 'always_visible': True}
            )
        ], style={'width': '400px'})
    ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'padding': '20px'}),

    # Metric Card for Anomaly Percentage
    html.Div([
        html.Div([
            html.H4("Anomaly Percentage", style={'textAlign': 'center', 'marginBottom': '10px'}),
            html.Div(id='total-anomalies', style={
                'fontSize': '32px',
                'fontWeight': 'bold',
                'textAlign': 'center',
                'padding': '20px',
                'backgroundColor': 'white',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'
            })
        ], style={'width': '300px', 'margin': 'auto'})
    ], style={'marginBottom': '20px'}),

    dcc.Tabs([
        dcc.Tab(label="Cycle Signals", children=[
            dcc.Graph(id="cycle-signals")
        ]),
        dcc.Tab(label="Anomaly Detection", children=[
            dcc.Graph(id="anomaly-detection")
        ]),
        dcc.Tab(label="Clustering", children=[
            dcc.Graph(id="clustering"),
            html.Div([
                html.H4("Clustering Information", style={'textAlign': 'center'}),
                html.P("This plot shows the signals grouped into clusters based on their features. " +
                      "Each point represents a window of the PPG signal, and colors indicate different clusters. " +
                      "Points that are close together have similar characteristics.")
            ], style={'padding': '20px'})
        ]),
        dcc.Tab(label="Classification Results", children=[
            dcc.Graph(id="classification"),
            html.Div([
                html.H4("Classification Information", style={'textAlign': 'center'}),
                html.P("The confusion matrix shows how well our model classifies Best vs Worst signals. " +
                      "The diagonal elements represent correct classifications, while off-diagonal elements show misclassifications.")
            ], style={'padding': '20px'})
        ])
    ])
])

@app.callback(
    [Output('cycle-signals', 'figure'),
     Output('total-anomalies', 'children')],
    [Input('group-selector', 'value'),
     Input('num-signals-slider', 'value')]
)
def update_raw_signals(selected_group, num_signals):
    fig = go.Figure()

    if selected_group == 'Best':
        selected_signals = best_signals[:num_signals]
    else:
        selected_signals = worst_signals[:num_signals]

    total_anomalies = 0
    total_windows = 0

    for filename in selected_signals:
        signal_data = normalize_signal(ppg_signals_df[filename]['cycle'].values)
        fig.add_trace(go.Scatter(
            y=signal_data,
            mode='lines',
            name=filename,
            opacity=1
        ))

        # Calculate anomalies for this signal
        signal_features = extract_features(signal_data, window_size, stride)
        total_windows += len(signal_features)

        scaled_features = scaler.transform(signal_features)
        predictions = iso_forest.predict(scaled_features)
        anomalies_in_signal = len(np.where(predictions == -1)[0])
        total_anomalies += anomalies_in_signal

    # Calculate percentage of anomalies
    anomaly_percentage = (total_anomalies / total_windows * 100) if total_windows > 0 else 0

    fig.update_layout(
        title=f"PPG Signals for {selected_group} Signals",
        xaxis_title="Sample",
        yaxis_title="Amplitude (normalized)",
        showlegend=True
    )

    return fig, f"{anomaly_percentage:.2f}% Anomalies"

# Rest of the callback functions remain the same as in the previous implementation
@app.callback(
    Output('anomaly-detection', 'figure'),
    [Input('group-selector', 'value'),
     Input('num-signals-slider', 'value')]
)
def update_anomaly_detection(selected_group, num_signals):
    fig = go.Figure()

    if selected_group == 'Best':
        selected_signals = best_signals[:num_signals]
    else:
        selected_signals = worst_signals[:num_signals]

    for filename in selected_signals:
        signal = normalize_signal(ppg_signals_df[filename]['cycle'].values)
        signal_features = extract_features(signal, window_size, stride)
        scaled_features = scaler.transform(signal_features)
        predictions = iso_forest.predict(scaled_features)
        anomaly_indices = np.where(predictions == -1)[0]

        # Plot the signal
        fig.add_trace(go.Scatter(
            y=signal,
            mode='lines',
            name=filename,
            opacity=1
        ))

        # Highlight anomalies
        for idx in anomaly_indices:
            point = idx * stride + window_size // 2
            if point < len(signal):
                fig.add_trace(go.Scatter(
                    x=[point],
                    y=[signal[point]],
                    mode='markers',
                    name=f'{filename} Anomaly',
                    marker=dict(color='red', size=10)
                ))

    fig.update_layout(
        title="Anomaly Detection",
        xaxis_title="Sample",
        yaxis_title="Amplitude",
        showlegend=True
    )
    return fig

@app.callback(
    Output('clustering', 'figure'),
    [Input('group-selector', 'value'),
     Input('num-signals-slider', 'value')]
)
def update_clustering(selected_group, num_signals):
    if selected_group == 'Best':
        selected_features = features_best
    else:
        selected_features = features_worst

    # Limit the features based on num_signals
    selected_features = selected_features[:num_signals*len(features_best)//len(best_signals)]

    # Perform clustering on selected features
    kmeans_local = KMeans(n_clusters=3, random_state=42).fit(selected_features)
    clusters_local = kmeans_local.labels_

    # Apply PCA for visualization
    pca_local = PCA(n_components=2)
    pca_features_local = pca_local.fit_transform(selected_features)

    fig = go.Figure()

    for cluster_num in np.unique(clusters_local):
        cluster_points = np.where(clusters_local == cluster_num)[0]

        fig.add_trace(go.Scatter(
            x=pca_features_local[cluster_points, 0],
            y=pca_features_local[cluster_points, 1],
            mode='markers',
            name=f'Cluster {cluster_num}',
            opacity=1
        ))

    fig.update_layout(
        title="Clustering Analysis",
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        showlegend=True
    )
    return fig

@app.callback(
    Output('classification', 'figure'),
    [Input('group-selector', 'value'),
     Input('num-signals-slider', 'value')]
)
def update_classification(selected_group, num_signals):
    # Use the trained classifier to make predictions
    y_pred = rf_classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    fig = go.Figure(data=go.Heatmap(
        z=cm,
        x=['Best', 'Worst'],
        y=['Best', 'Worst'],
        colorscale='Blues',
        opacity=1
    ))

    fig.update_layout(
        title="Classification Results (Confusion Matrix)",
        xaxis_title="Predicted",
        yaxis_title="Actual"
    )
    return fig

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)